In [15]:
#Packages
import pandas as pd
import numpy as np

In [16]:
#Import dataset
file_path = 'C:\\Users\\Matt\\Documents\\Python code thesis\\Datasets\\customersat.csv'
customersat = pd.read_csv(file_path)

In [17]:
#Preprocessing steps

#All observations are independent
len(customersat.loc[:,"ID"].unique())==len(customersat.loc[:,"ID"])

#Dropping ID
customersat = customersat.drop('ID',axis=1).copy()

#No NA values
np.any(customersat.isnull())

#Checking if any columns are duplicated
duplicated_columns = customersat.transpose().duplicated()
customersat = customersat.loc[:,np.logical_not(duplicated_columns)]

#Identifying negative values
def neg_check(x):
    return(np.any(x<0))
ind_col_neg = np.apply_along_axis(neg_check, axis=0, arr=customersat)
dat_neg = customersat.loc[:,ind_col_neg]

def sum_neg(x):
    return(sum(x<0))
num_neg = np.apply_along_axis(sum_neg, axis=0, arr=dat_neg)
num_neg = pd.DataFrame({'Variable':dat_neg.columns.tolist(), '# negatives': num_neg})
num_neg = num_neg.sort_values('# negatives')
num_neg = num_neg.reset_index(drop=True)
num_neg

#Removing observations columns have less than 122 negative values
ind_neg = {}
for i in range(num_neg.shape[0]-2):
    feat_name = num_neg.loc[i,'Variable']
    ind_neg[feat_name] = list(np.where(customersat.loc[:,feat_name]<0)[0])
ind_neg = list(ind_neg.values())
ind_neg = [num for sublist in ind_neg for num in sublist]
ind_neg_row = list(set(ind_neg))
customersat = customersat.drop(ind_neg_row,axis=0)
customersat = customersat.reset_index(drop=True)

#Creating indicator variables for columns with more than 1 000 negative values
ind_col_neg = np.apply_along_axis(neg_check, axis=0, arr=customersat)
dat_neg = customersat.iloc[:,ind_col_neg]
dat_neg_int = pd.DataFrame()
column_names = [dat_neg.columns[0]+'_ind',dat_neg.columns[1]+'_ind']
for i in range(dat_neg.shape[1]):
    x_int = []
    for j in dat_neg.iloc[:,i]:
        if (j<0):
            x_int.append(1)
        else:
            x_int.append(0)
    dat_neg_int[column_names[i]] = x_int
    
#Setting negative values to zero and attaching indicator variables
customersat[customersat <0] =0
customersat = pd.concat([customersat, dat_neg_int], axis=1)

#Identifying some features that have element 9999999999
def max_vals(x):
    return([max(x),sum(max(x)==x)])
max_cust = np.apply_along_axis(max_vals, axis=0, arr=customersat)

max_summary = pd.DataFrame({'Variable':customersat.columns.tolist(), 'Max values':max_cust[0], 'Number of time occured':max_cust[1]})
pd.set_option('display.max_rows', None)
np.set_printoptions(suppress=True)
max_summary.sort_values('Max values')

#Removing observations that have 9999999999
row_miss = np.any(customersat==9999999999, axis=1)
customersat = customersat[-row_miss]

#Remove columns which are constant
rep_ind = []
for i in range(customersat.shape[1]):
    rep_ind.append(len(np.unique(customersat.iloc[:,i]))!=1)
customersat = customersat.loc[:,rep_ind]